## CART Tree

1. Aconcdicionar dataset
    1. Identificar acciones frecuentes para  cada session y cada usuario
    2. Datos a array
    3. Generar clases 
2. Setear el gridsearch **-->** Quedarse con el mejor modelo 
3. identificar reglas

1. https://pythonprogramminglanguage.com/decision-tree/
2. http://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py
3.  http://scikit-learn.org/stable/modules/grid_search.html#grid-search
4. http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
5. http://scikit-learn.org/stable/modules/tree.html#tree
6. http://dataaspirant.com/2017/02/01/decision-tree-algorithm-python-with-scikit-learn/


In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

# 1. Preparacion del dataset

In [2]:
file_salida = 'C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\base_con_clusters.csv'

In [3]:
df = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\all_base.csv').set_index('session_fake_id')

In [4]:
base_lift = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\base_itemsets_lift_1.csv')

In [5]:
base_cluster = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\clustering_vector.csv'
                           , header = None, names = ['itemset', 'cluster'])
for i in range(3):
    base_cluster['c{}'.format(i+1)] = 1*(base_cluster['cluster'] ==(i+1))

In [6]:
itemsets_series = base_lift['itemsets'].apply(lambda x: x.strip("[]").replace("'", "").replace(" ", "").split(","))
base_lift.pop('itemsets')
base_lift = pd.concat([base_lift,itemsets_series],axis = 1)

In [7]:
# construyo una matrix con una fila por cada valor de accion y una columna por cada canasta, con unos y ceros por cada una  

acciones = list(df.columns[11:35]) # uso solo hasta la 35 porque el resto no esta en las canastas frequentes
base_lift_acciones = []

for i , accion in enumerate(acciones):
    for j, canasta in enumerate(itemsets_series):
            base_lift_acciones.append(1 * str(accion) in canasta)
            
base_lift_acciones = np.array(base_lift_acciones).reshape(len(acciones), itemsets_series.shape[0]) * 1



In [8]:
#vector con el total de acciones de cada canasta
num_acciones_canasta = base_lift_acciones.sum(axis=0)

In [9]:
#a partir de esto hay que hacer un loop por las filas de df (en bines) y hacer append en un archivo
for v in range(1 ,15):
    if v == 14:
        df_sub = df.iloc[ ((v-1)*100000):,:]
    else:
        df_sub = df.iloc[ ((v-1)*100000):((v)*100000) ,:]
    
    df_a_0 = df_sub.iloc[:,11:35].dot(base_lift_acciones).divide(num_acciones_canasta, axis = 1).apply(lambda x: (x==1)*1)
    df_c_0 = df_a_0.dot(base_cluster.iloc[:,2:]).apply(lambda x: (x !=0)*1)
    df_a_0.columns = [ 'i' + str(i +1) for i in df_a_0.columns ]

    df_0 = pd.concat([ df_sub, df_a_0, df_c_0], axis = 1)
    
    df_0['clase'] = df_0['starter'] + df_0['finisher']
    
    if v == 1:
        df_0.to_csv(file_salida, mode = 'w')
    else:
        df_0.to_csv(file_salida, header = False , mode = 'a')

